# 第4章: 形態素解析

夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](https://nlp100.github.io/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．


In [1]:
!wget https://nlp100.github.io/data/neko.txt

--2023-02-04 20:17:44--  https://nlp100.github.io/data/neko.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975789 (953K) [text/plain]
Saving to: ‘neko.txt’

neko.txt            100%[===================>] 952.92K  --.-KB/s    in 0.06s   

2023-02-04 20:17:45 (16.1 MB/s) - ‘neko.txt’ saved [975789/975789]



In [2]:
!sudo apt install mecab libmecab-dev mecab-ipadic-utf8
!mecab neko.txt -o neko.txt.mecab

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 27 not upgraded.
Need to get 7,373 kB of archives.
After this operation, 60.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libmecab2 amd64 0.996-10build1 [233 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libmecab-dev amd64 0.996-10build1 [285 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 mecab-utils amd64 0.996-10build1 [4,912 B]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 mecab-ipadic all 2.7.0-20070801+main-

## 30. 形態素解析結果の読み込み
---
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [52]:
morphemes_list = []
morphemes = []
with open('./neko.txt.mecab', 'r') as f:
    for line in f:
        if line == 'EOS\n':
            morphemes_list.append(morphemes)
            morphemes = []
        elif len(line) > 1:
            surface, features = line.split('\t')
            features = features.split(',')
            morphemes.append({
                'surface': surface,
                'base': features[6],
                'pos': features[0],
                'pos1': features[1],
            })

for morphemes in morphemes_list[2]:
    print(morphemes)

{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}
{'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}
{'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}
{'surface': '', 'base': '*\n', 'pos': '記号', 'pos1': '一般'}


## 31. 動詞Permalink
---
動詞の表層形をすべて抽出せよ．

In [50]:
verb_surfaces = set()
for morphemes in morphemes_list:
    for morpheme in morphemes:
        if morpheme['pos'] == "動詞":
            verb_surfaces.add(morpheme['surface'])

print(f"動詞の表層形の種類: {len(verb_surfaces)}\n")
for verb_surface in list(verb_surfaces)[:10]:
    print(verb_surface)

動詞の表層形の種類: 3893

あき
断わっ
のべる
入り込ん
洗い
凌ぐ
待つ
くらん
もらわ
こし


## 32. 動詞の基本形
---
動詞の基本形をすべて抽出せよ．

In [53]:
verb_bases = set()
for morphemes in morphemes_list:
    for morpheme in morphemes:
        if morpheme['pos'] == "動詞":
            verb_bases.add(morpheme['base'])

print(f"動詞の基本形の種類: {len(verb_bases)}\n")
for verb_base in list(verb_bases)[:10]:
    print(verb_base)

動詞の基本形の種類: 2300

盛り込む
せしめる
漕ぎつける
のべる
凌ぐ
待つ
見透かす
見向く
受け取れる
張り切れる


## 33. 「AのB」
---
2つの名詞が「の」で連結されている名詞句を抽出せよ．

## 34. 名詞の連接
---
名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

## 35. 単語の出現頻度
---
文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

## 36. 頻度上位10語
---
出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## 37. 「猫」と共起頻度の高い上位10語
---
「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## 38. ヒストグラム
---
単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．

## 39. Zipfの法則
---
単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．